In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd
import sklearn as sk
import sys
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RANSACRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.datasets import load_boston

# Checamos versiones
print(np.__version__)
print(sns.__version__)
print(pd.__version__)
print(sk.__version__)
print(sys.version)

# Se carga base de datos Boston Housing Bubble
'''
boston_dataset = load_boston()
bostonData = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
print(boston_dataset.keys())
print(boston_dataset.DESCR)
print(bostonData)

# EDA
print(boston_dataset.feature_names[0])
SelectedColumns = boston_dataset.feature_names
sns.pairplot(boston_dataset.feature_names, height=1.5)
plt.show()
'''

# Se carga datos provientes de Boston Housing Buble CSV con pandas

filePath = "C:/Users/Armando/Documents/DataScience/DataSets/housing.csv"

encabezados = ['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'b', 'lstat', 'medv']

#df = pd.read_csv(filePath)
#df = load_boston()

data_url = "http://lib.stat.cmu.edu/datasets/boston"
df = pd.read_csv(data_url, sep="\s+", skiprows=22, names=encabezados)


print(df.head())

X = np.hstack([df.values[::2, :], df.values[1::2, :2]])
y = df.values[1::2, 2]
print(X)
print(y)
pd.options.display.float_format = '{:,.3f}'.format      # Formato para únicamente tomar 3 decimales

# -------------------------------- 1. Observación y depuración de datos -------------------------------------
# print(df)
df.head()
df.describe()
#print(df.shape)
#print(df.dtypes)
#print(pd.isnull(df))

# ------------------------------------------- 2. EDA --------------------------------------------------------------
col_study = ['crim', 'zn', 'indus', 'nox', 'rm']
#sns.pairplot(df[col_study], height=2.5)
#plt.show()

#print(df.corr())       # Muestra correlaciones de los datos

# Correlogramas con estilos
plt.figure(figsize=(16,10))
sns.heatmap(df[['crim', 'zn', 'indus', 'chas', 'medv']].corr(), annot=True)
plt.show()



# ------------------------------------------ 3. Linear Regression -----------------------------------------------
# a) Se establecen valores en variables
xVariable = 'rm'
yVariable = 'medv'
X = df[xVariable].values.reshape(-1, 1) # reshape lo que hace es mostrar registros por columna. Sin reshape muestra datos horizontalmente
y = df[yVariable].values

#print(y)

# b) Se escoge tipo de modelo
model = LinearRegression()      # Se pueden especificar hiperparámetros

# c) Ajustamos modelo a los datos
model.fit(X, y)
a = model.coef_
b = model.intercept_

#print(a)
#print(b)

plt.figure(figsize=(12, 10))
sns.regplot(X, y)
plt.xlabel("average number of rooms per dwelling")
plt.ylabel("Median value of owner-occupied homes in $1000's")
plt.show()

# 3. ------------------------------- Pruebas de especificación ---------------------------------------------
# Normalidad (Gráfica de distribución de los datos. Debe ser normal)
sns.jointplot(x=xVariable, y=yVariable, data=df, kind='reg', height=10)
plt.show()

# RANSAC Iteración para determinar inliers, outliers y ajustar la regresión. Correción de No Normalidad, heteroscedasticidad y autocorrelación
# No tomando en cuenta los outliers la regresión debería estar correctamente especificada
ransac = RANSACRegressor()
ransac.fit(X, y)

inlier_mask = ransac.inlier_mask_
outlier_mask = np.logical_not(inlier_mask)

line_x = np.arange(3, 10, 1)       # crea arreglo [3,4,5,6,7,8,9] ya que la regresión original abarcaba estos valores en x
line_y_ransac = ransac.predict(line_x.reshape(-1, 1))

sns.set(style='darkgrid', context='notebook')
plt.figure(figsize=(12, 8))
plt.scatter(X[inlier_mask], y[inlier_mask], c='blue', marker='o', label='Inliers')
plt.scatter(X[outlier_mask], y[outlier_mask], c='brown', marker='s', label='Outliers')
plt.plot(line_x, line_y_ransac, color='red')
plt.xlabel('average number of rooms per dwelling')
plt.ylabel("Median value of owner-occupied homes in $1000's")
plt.legend(loc="upper left")
plt.show()

a_corrected = ransac.estimator_.intercept_
b_corrected = ransac.estimator_.coef_

#print(a_corrected)
#print(b_corrected)


# 4. -------------------------------- Predicción -------------------------------------------------------
'''
#Pronóstico espurio
x_ahead = np.array([5]).reshape(1, -1)
predict = model.predict(x_ahead)    # Pronóstico de y en un periodo posterior
print(predict)

'''

#Pronóstico correctamente especificado
x_ahead = np.array([5]).reshape(1, -1)
print(x_ahead)
robustPredict = ransac.predict(x_ahead)
print(robustPredict)



# ----------------------------------- Evaluación desempeño del modelo de regresión (Propio de ML) -----------------------------------------------
# a) Pruebas
# b) Entrenamiento


print("Evaluaciòn del desempeño del modelo")
X = df.iloc[:, :-1].values      # Obtiene todas las columnas excepto la última que es 'medv'
y = df[yVariable].values

#print(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#print("X de entrenamiento")
#print(X_train)
#print("X de pruebas")
#print(X_test)
#print("y de entrenamiento")
#print(y_train)
#print("y de pruebas")
#print(y_test)

lr = LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

lr.fit(X_train, y_train)

y_train_pred = lr.predict(X_train)
y_test_pred = lr.predict(X_test)

# En ML se manejan tres pruebas para validar y probar la robustez del modelo sin embargo. En series de tiempo no va a ser suficiente.
# Nosotros aplicaremos todas las pruebas de especificación aprendidas más las pruebas de precisión de los pronósticos.
# a) MSE, b) MAE, c) RSME

# Método 1 Análisis (visual) Residuos
plt.figure(figsize=(12,8))
plt.scatter(y_train_pred, y_train_pred - y_train, c='blue', marker='o', label='Training Data')
plt.scatter(y_test_pred, y_test_pred - y_test, c='orange', label='Test Data')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.legend(loc='upper left')
plt.hlines(y=0, xmin=-10, xmax=50, lw=2, color='k')
plt.xlim([-10, 50])
plt.show()

# Método 2. Mean Squared Error (MSE)

mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
#print(mse_train)
#print(mse_test)

# Método 3. R2

R2_train = r2_score(y_train, y_train_pred)
R2_test = r2_score(y_test, y_test_pred)

#print(R2_train)
#print(R2_test)